本文学习于论文《An Introduction to Conditional Random Fields》,作者:Charles Sutton 和 Andrew McCallum

# 摘要: Abstract

通常，我们希望预测大量彼此依赖的变量以及在其它可观测变量上预测变量。结构化预测方法本质上是分类和图形建模的组合，结合了图形建模对多变量数据进行紧凑建模的能力，以及分类方法使用大量输入特征进行预测的能力。本教程描述了条件随机场CRFs，它是一种流行的结构化预测的概率方法。CRFs在自然语言处理、计算机视觉和生物信息学中得到了广泛的应用。我们描述了用于CRFs的推理和参数估计的方法，包括用于实现大规模CRFs的实际问题。我们不假设读者知晓先前的图形建模知识，因此本教程旨在对各种领域的从业者有用。

# 引言: Introduction

许多应用程序的基础是能够**预测彼此依赖的多个变量**。这样的应用程序就像分类一张图片的区域，估计Go游戏中的得分，分割DNA链中的基因，以及从自然语言文本中提取语法一样多样化。在这样的应用中，我们希望，在给定可观测特征向量$**x**$情况下，预测随机向量$y = \left\{y_{0},y_{1},...,y_{T}\right\}$。来自自然语言处理的一个相对简单的例子是词性标注POS，其中每个变量$y_{s}$是位置**s**处单词的词性标签，并且输入$x$被分割成特征向量$\left\{x_{0},x_{1},...,x_{T}\right\}$。每个$x_{s}$处包含有关位置**s**处单词的各种信息，例如其标识、前缀和后缀等拼写特征、域特定词典中的成员身份以及语义数据库(如WordNet)中的信息。

这种**多变量预测问题**的一种方法，特别是如果我们的目标是最大化正确分类的标签$y_{s}$的数量，就是为每个位置**s**学习一个独立的位置分类器，它使$X \mapsto y_{s}$。然而，困难在于**输出变量具有复杂的依赖性**。例如，文档中的相邻单词或图像的相邻区域倾向于具有相似的标签。或者输出变量可以表示复杂的结构，例如解析树，其中选择在树顶部附近使用的语法规则会对树的其余部分产生很大影响。

一种很自然的表示输出变量相互依赖的方式由图模型提供。图模型--包括贝叶斯网络、神经网络、因子图、马尔科夫随机场、伊辛模型和其它组成的多种模型族--表示多个变量的复杂分布为局部因子在变量的较小子集上的乘积。然后可以描述概率密度的给定因式分解(p(a,b,c)=p(a)p(b,c|a)p(b|a)p(c|b,a))如何对应于分布所满足的特定的条件独立关系的集合。这种对应使得建模更加方便，因为我们对领域的知识通常表明了合理的条件独立性假设，这然后决定我们选择的因素。

在图形模型的学习方面，特别是在统计自然语言处理方面，很多工作都集中在生成模型上，这些模型试图明确地对输入和输出的联合概率分布$p(y;x)$进行建模。尽管这种有优点，但它也有重要的局限性。不仅$x$的维度可能会非常大，而且$x$的特征之间具有复杂的依赖性，因此很难在$x$上构建概率分布。对输入特征之间的依赖关系进行建模可能会导致难以处理的模型，但忽略它们会导致性能降低。

该问题的解决方案是直接对条件分布$p(y|x)$建模，这是分类所需的全部。这便是一个条件随机场CRFs。CRFs本质上是一种结合分类和图形建模优势的方法，将对多变量数据紧凑建模的能力与利用大量输入特征进行预测的能力相结合。条件模型的优点是仅涉及$x$中的变量的依赖关系，在条件模型中不起作用，因此精确的条件模型可以具有比联合模型更简单的结构。因此，条件模型和CRF之间的差异完全类似于朴素贝叶斯和逻辑回归分类器之间的差异。实际上，多项式逻辑回归模型可以看作是最简单的CRF，其中只有一个输出变量。

有很多对CRFs应用的兴趣，成功的应用包括文本处理，生物信息学和计算机视觉。尽管CRFs的早期应用使用线性链，但CRF的最近应用也使用了更通用的图形结构。通用的图形结构对预测复杂结构(如图形和树)以及放宽实体之间的iid假设非常有用，如关系学习。

本教程介绍了使用CRFs进行建模、推理和参数估计的方法。我们不假设读者知晓先前的图形建模知识，因此本教程旨在对各种领域的从业者有用。我们首先描述CRFs中的建模问题，包括线性链CRFs、通用图形结构的CRFs和包含潜在变量的隐藏CRFs。我们描述了如何将CRFs视为众所周知的逻辑回归的一般化，以及作为隐马尔可夫模型的判别模拟。

在接下来的两章，我们描述了CRF中的推理和学习。这两个过程是紧密耦合的，因为学习通常把推理视为一个子程序。虽然我们讨论的推理算法是图形模型的标准算法，但推理嵌入在外部参数估计过程中的事实引发了其它问题。最后，我们讨论了CRFs与其它模型族之间的关系，包括其它结构化预测方法，神经网络和最大熵马尔科夫模型。

## 实现细节: Implementation Details

在整篇专著中，我们试图指出有时在研究文献中被忽略的实现细节。例如，我们讨论与特征工程相关的问题，避免推理期间的数值溢出，以及CRFs训练在某些基准熵的可扩展性。

由于这是我们关于实现细节的第一部分，因此提及一些可用的CRFs实现似乎是恰当的。在撰写本文时，一些流行的实现是:

|包|地址|
|:--:|:--:|
|CRF++|http://crfpp.sourceforge.net/ |
|MALLET|http://mallet.cs.umass.edu/ |
|GRMM|http://mallet.cs.umass.edu/grmm/ |
|CRFSuite|http://www.chokkan.org/software/crfsuite/ |
|FACTORIE|http://www.factorie.cc |

此外，马尔科夫逻辑网络的包(例如，Alchemy: http://alchemy.cs.washington.edu/ )可用于构建CRFs模型。Alchemy，GRMM和FACTORIE是我们知道的处理任意图形结构的唯一工具包。

# 建模: Modeling

在本章中，我们从建模角度描述条件随机场，解释CRFs如何**表示结构化输出上的分布作为高维输入向量的函数**。CRF既可以理解为逻辑回归分类器到任意图形结构的扩展，也可以理解为结构化数据的生成模型的判别模拟，例如隐马尔可夫模型。

我们首先简要介绍图形建模以及NLP中年生成和判别模型的描述。接着，我们便能够展示条件随机场的正式定义，既适用于线性链的常用情况，也适用于一般的图形结构。最后，我们提供了一些在应用程序中如何使用不同结构的示例，以及一些有关特征工程的实现细节。

## 图形建模: Graphical Modeling

图形建模是多变量概率分布中表示和推理的一个强大框架。事实证明，它在随机建模的各个领域都很有用，包括编码理论，计算机视觉，知识表示，贝叶斯统计和自然语言处理。

许多变量的分布朴素地表示可能是昂贵。例如，$n$ 个二进制分类变量的联合概率表需要存储 $O(2^{n})$个浮点数。**图形建模视角的洞察力是，许多变量的分布通常可以表示为局部函数的乘积，每个函数都依赖于更小的变量子集**。这种因式分解与变量之间的某些条件独立关系密切相关--两种类型的信息都可以通过图轻松汇总。实际上，**因式分解**、**条件独立**和**图形结构**之间的这种关系构成了图形建模框架的大部分功能：条件独立视角对于设计模型最有用，而因式分解视角对于设计推理算法最有用。

在本节的其余部分，我们从因式分解和条件独立视角介绍图形模型，重点关注那些基于无向图的模型。Koller和Friedman在教科书(Daphne Koller and Nir Friedman《Probabilistic Graphical Models:Principles and Techniques》)中提供了关于图形建模和近似推理的更详细的现代观点。

### 无向模型: Undirected Models

我们考虑随机变量集合$V = X \cup Y$ 上的概率分布，$X$ 是输入变量的集合，它是可观测的；$Y$ 是输出变量的集合，它是我们期望预测的。每个变量 $s \in V$ 都取自集合 $V$ 的结果，可以是连续的，也可以说离散的，尽管我们在此教程中只考虑离散情况。对于 $X$ 的任意赋值由向量 $x$ 表示。给定变量 $s \in V$ ，符号 $x_{s}$表示由 $x$ 分配给 $s$ 的值，并且类似地，符号 $x_{a}$ 表示由 $x$ 分配给子集 $a \subset X$ 的值。符号 $1_{x=x^{'}}$ 表示 $x$ 的指示函数，当 $x=x^{'}$ 时取1，否则取0。我们还要求边缘化的符号。对于固定变量赋值$y_{s}$，我们使用求和符号 $\sum_{y \y_{s}}$来表示在所有可能赋值 $y$ 上的求和，其变量 $s$ 的值等于 $y_{s}$ 。

假设我们认为概率分布 $p$ 可以用 $\Psi_{a}(x_{a},y_{a})$ 形式的因子乘积来表示，其中每个因子的范围为 $a \subseteq V$。这种因式分解可以允许我们更有效地表示 $p$，因为集合 $a$ 可以比完整变量集合 $V$ 小得多。我们假设不失一般性，每个不同的集合 $a$ 最多只有一个因子 $\Psi_{a}$。

无向图模型是一个概率分布族，根据给定的范围集合进行分解。正式地说，给定一组子集 $F=a \subset V$,无向图模型被定义为可以写成如下形式的所有分布的集合:

$$p(x,y) = \frac{1}{Z}\prod_{a\in F}\Psi_{a}(x_{a},y_{a}), (2.1)$$

对于任意的局部函数 $F = \left\{\Psi_{a}\right\}$,有$\Psi_{a}:V^{|a|} \rightarrow R^{+}$。(这些函数也称为因子或兼容性函数)我们有时候会用**术语"随机场"来指那些由无向图模型定义的分布**。当我们讨论如何将(2.1)的因式分解表示为图时，术语"图形模型"的原因很快就会变得明显。

常数 $Z$ 是一个标准化因子，它确保分布 $p$ 等于1.它定义如下:

$$Z = \sum_{x,y}\prod_{a\in F}\Psi_{a}(x_{a},y_{a}), (2.2)$$

数量 $Z$ ,作为因子集 $F$的函数，有时被称为配分函数。注意，(2.2)中的求和是在指数上对 $x$ 和 $y$ 的任何可能的分配。出于这个原因，计算 $Z$ 一般是难以处理的，但是关于如何近似它的研究有很多。

### 有向模型: Directed Models

## 生成模型和判别模型: Generative versus Discriminative Models

## 线性链条件随机场: Linear-chain CRFs

## 通用的条件随机场: General CRFs

## 条件随机场的应用: Applications of CRFs

## 特征工程: Feature Engineering

## 术语的说明: Note on Terminology

# Inference

## Linear-Chain CRFs

## Inference in Graphical Models

## Implementation Concerns

# Parameter Estimation

## Maximum Likelihood

## Stochastic Gradient Methods

## Parallelism

## Approximate Training

## Implementation Concerns